In [1]:
from glob import glob
from streaming import MDSWriter
from streaming import StreamingDataset, LocalDataset
from transformers import default_data_collator, DataCollatorForLanguageModeling
from tqdm import tqdm
import numpy as np

/home/ubuntu/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [4]:
folders = sorted(glob('tokenized_extra/tokenized-*'), key = lambda x: int(x.split('-')[-1]))

In [5]:
from streaming.base.format.mds.encodings import Encoding, _encodings

class UInt16(Encoding):
    def encode(self, obj) -> bytes:
        return obj.tobytes()

    def decode(self, data: bytes):
        return np.frombuffer(data, np.uint16)

_encodings['uint16'] = UInt16

In [6]:
columns = {
    'input_ids': 'uint16',
}

compression = 'zstd'
hashes = 'sha1', 'xxh64'

In [9]:
!rm -rf combine-extra

In [10]:
with MDSWriter(out='combine-extra', columns=columns, compression=None, hashes=hashes) as out:
    for no, f in enumerate(folders):
        print(no)
        try:
            dataset = StreamingDataset(local=f)
            for i in tqdm(range(len(dataset))):
                out.write(dataset[i])
        except Exception as e:
            print(e)
            pass

0


100%|██████████| 21625/21625 [00:03<00:00, 7075.24it/s] 


1


100%|██████████| 21948/21948 [00:04<00:00, 5343.42it/s]


2


...

100%|██████████| 22024/22024 [00:03<00:00, 5801.87it/s]


11


100%|██████████| 20130/20130 [00:03<00:00, 6466.74it/s]


12


100%|██████████| 17117/17117 [00:03<00:00, 4814.50it/s]


In [11]:
!du -hs combine-extra

2.2G	combine-extra


In [12]:
dataset = LocalDataset('combine-extra')

In [13]:
for i in tqdm(range(len(dataset))):
    dataset[i]

100%|██████████| 278433/278433 [00:28<00:00, 9802.88it/s] 
